In [28]:
from pprint import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import warnings
warnings.filterwarnings('ignore') # We can suppress the warnings

In [17]:
pip install praw

Note: you may need to restart the kernel to use updated packages.


In [18]:
import praw

In [19]:
user_agent = "San 1.0 by/u/Sandy-Mer96"
reddit = praw.Reddit(
    client_id="reVssLmWomqdowekRVQuLA",
    client_secret="3WUuSJjdoT6uTGtDtmEcyC0IgPq-7Q",
    user_agent=user_agent
)

In [20]:
# hot new rising top

headlines = set()
for submission in reddit.subreddit('HousingIreland').hot(limit=None):
    headlines.add(submission.title)
print(len(headlines))

452


In [24]:
Housing_Ireland = pd.DataFrame(headlines)
Housing_Ireland.head()

,0
0,Councillors query long-term lease deal for 124...
1,From Affordable Housing to Vulture Funds: An A...
2,So when are we pitching tents on the grounds o...
3,Northern Ireland Letting - Is criminal record ...
4,Affordable housing Bill 450K - Dan’s mix tape


In [25]:
Housing_Ireland.to_csv("Housing.csv", header=False, encoding="utf-8", index=False)

In [29]:
Housing_Ireland=pd.read_csv('Housing.csv')


In [119]:
Housing_Ireland.head()

,comments
0,Councillors query long-term lease deal for 124...
1,From Affordable Housing to Vulture Funds: An A...
2,So when are we pitching tents on the grounds o...
3,Northern Ireland Letting - Is criminal record ...
4,Affordable housing Bill 450K - Dan’s mix tape


In [103]:
Housing_Ireland = pd.DataFrame(headlines, columns=['comments'])
Housing_Ireland.head()

,comments
0,Councillors query long-term lease deal for 124...
1,From Affordable Housing to Vulture Funds: An A...
2,So when are we pitching tents on the grounds o...
3,Northern Ireland Letting - Is criminal record ...
4,Affordable housing Bill 450K - Dan’s mix tape


In [41]:
Housing_Ireland.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452 entries, 0 to 451
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   comments  452 non-null    object
dtypes: object(1)
memory usage: 3.7+ KB


## Data Pre Processing

In [42]:
# Drop records for the column named as "comments" where confidence is below 0.5
Housing_Ireland_filtered = Housing_Ireland.drop(Housing_Ireland[Housing_Ireland['comments'].str.len() < 0.5].index, axis=0)

# Display the rows and columns of the filtered DataFrame
Housing_Ireland_filtered.shape


(452, 1)

In [45]:
# Store the column of the dataframe named as "comments"
X = Housing_Ireland['comments']

# Display the values in the "comments" column
print(X)


0      Councillors query long-term lease deal for 124...
1      From Affordable Housing to Vulture Funds: An A...
2      So when are we pitching tents on the grounds o...
3      Northern Ireland Letting - Is criminal record ...
4          Affordable housing Bill 450K - Dan’s mix tape
                             ...                        
447                         Rebuilding ireland home loan
448    As Pandemic Evictions Rise, Spaniards Declare ...
449    Couldn't believe what I was hearing from a FF ...
450    Good morning to @Paschald & @DarraghOBrienTD f...
451       my global housing market copypasta & link dump
Name: comments, Length: 452, dtype: object


## Clean Your Text Data

In [47]:
from nltk.corpus import stopwords
nltk.download('stopwords')

import string
from nltk.stem import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sandr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [48]:
# Store the stopwords into the object named as "stop_words"
stop_words = stopwords.words('english')

# Store the string.punctuation into an object punct
punct = string.punctuation

# Initialise an object using a method PorterStemmer
stemmer = PorterStemmer()

In [49]:
import re

cleaned_data = []

for i in range(len(X)):
    comment = re.sub('[^a-zA-Z]', ' ', X.iloc[i])  # Remove non-alphabetic characters from the comment
    comment = comment.lower().split()  # Convert comment to lowercase and split into individual words
    comment = [stemmer.stem(word) for word in comment if (word not in stop_words) and (word not in punct)]  # Apply stemming and remove stop words and punctuation
    comment = ' '.join(comment)  # Join the processed words back into a string
    cleaned_data.append(comment)  # Add the cleaned comment to the list



In [50]:
# Display the cleaned_data
cleaned_data

['councillor queri long term leas deal apart car park former regenc hotel',
 'afford hous vultur fund z ish ireland hous crisi',
 'pitch tent ground oireachta',
 'northern ireland let crimin record check allow landlord',
 'afford hous bill k dan mix tape',
 'hous price increas last year',
 'lone parent children account half homeless famili',
 'protest galway saturday',
 'trauma child homeless',
 'updat protest',
 'worth watch rori hearn speak nation hous emerg',
 'good sourc advic new hous',
 'hous crisi state back invest fund leas home local author lucr deal',
 'anybodi luck queer hous ireland',
 'higher board approv landlord chang rent price ireland',
 'ff fg green vote tax break cuckoo fund reward snatch famili home govern turn back peopl',
 'mother god hate peopl',
 'world famou essay jonathan swift propos solut irish poverti relev today year ago',
 'slightli older crowd might find interest',
 'dumb idea',
 'council swap dublin k offer swap council hous',
 'thought would interest g

In [51]:
print(X)

0      Councillors query long-term lease deal for 124...
1      From Affordable Housing to Vulture Funds: An A...
2      So when are we pitching tents on the grounds o...
3      Northern Ireland Letting - Is criminal record ...
4          Affordable housing Bill 450K - Dan’s mix tape
                             ...                        
447                         Rebuilding ireland home loan
448    As Pandemic Evictions Rise, Spaniards Declare ...
449    Couldn't believe what I was hearing from a FF ...
450    Good morning to @Paschald & @DarraghOBrienTD f...
451       my global housing market copypasta & link dump
Name: comments, Length: 452, dtype: object


In [100]:
# Collect all columns into dataframe named as sentiment_ordering
sentiment_ordering = ['negative', 'neutral', 'positive']

# Store the column of the dataframe named as "comments" into variable y
X = Housing_Ireland['comments']

# Apply the lambda function to convert sentiment values to corresponding indices
X = X.apply(lambda x: sentiment_ordering.index(x) if x in sentiment_ordering else -1)



In [101]:
X.head()

0   -1
1   -1
2   -1
3   -1
4   -1
Name: comments, dtype: int64

## Bag of Words using CountVectorizer

In [61]:
from sklearn.feature_extraction.text import CountVectorizer

# Instantiate an object cv by calling a method named as CountVectorzer()
cv    = CountVectorizer(max_features =3000, stop_words = ['Paschald', 'DarraghOBrienTD'])

# Train the dataset by calling a fit_transform() method
X_fin = cv.fit_transform(cleaned_data).toarray()

# Display the rows and colums
X_fin.shape

(452, 1358)

In [62]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

# Instantiate an object model by calling a method MultinomialNB()
model = MultinomialNB()

In [63]:
# Split the dataset into training and testing parts
X_train, X_test, y_train, y_test = train_test_split(X_fin, y, test_size = 0.3)

In [64]:
# Train the model by calling a method fit()
model.fit(X_train,y_train)

MultinomialNB()

In [66]:
# Call predict() method
y_pred = model.predict(X_test)

In [67]:
from sklearn.metrics import classification_report

# Instantiate a mthod named as Cla
cf = classification_report(y_test, y_pred)

# Display the values of an object cf
print(cf)

                                                                                                                                                                                                                                                                                                    precision    recall  f1-score   support

                                                                                                                                                                                                                                     "Housing For All" AKA re-arranging deck chairs on the Titanic       0.00      0.00      0.00       1.0
                                                                                                                                                                                                                                        "Ireland builds houses at the wrong time, place and price"       0.00      0.00      0.00       0.0
   

## Feature Generation using TF-IDF

In [68]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create an object 'tf' by calling a method TfidfVectorizer()
tfidf = TfidfVectorizer(max_features =3000, stop_words = ['Paschald', 'DarraghOBrienTD'])

# Train the dataset by cjalling a method fit_tranform() 
X_tfidf = tfidf.fit_transform(cleaned_data).toarray()

In [69]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

# Instantiate an object model by calling a method MultinomialNB()
model_tdidf = MultinomialNB()

In [70]:
# Split the dataset into training and testing parts
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size = 0.3)

In [71]:
# Train the model by calling a method fit()
model_tdidf.fit(X_train, y_train)

MultinomialNB()

In [72]:
# Call predict() method
y_pred = model_tdidf.predict(X_test)

In [73]:
from sklearn.metrics import classification_report

# Instantiate a mthod named as Cla
cf = classification_report(y_test, y_pred)

# Display the values of an object cf
print(cf)

                                                                                                                                                                                                                                                                                                       precision    recall  f1-score   support

                                                                                                                                                                                                                                           "Ireland builds houses at the wrong time, place and price"       0.00      0.00      0.00       0.0
                                                                                                                                                  "Renters who buy – even at elevated prices – will see their housing costs plunge" - Irish Times advocating for loosening Central bank lending rules       0.00      0.00      0.00     


## VADER

In [104]:
pip install vaderSentiment

Note: you may need to restart the kernel to use updated packages.


In [105]:
# Load thelibraries
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Create and initialise an object
sentiment = SentimentIntensityAnalyzer()
text_1 = "Ireland builds houses at the wrong time, place and price."
text_2 =  "Am I the only one completely deflated and depressed by the Irish housing market?"

sent_1 = sentiment.polarity_scores(text_1)
sent_2 = sentiment.polarity_scores(text_2)

print("Sentiment of text 1:", sent_1)
print("Sentiment of text 2:", sent_2)

Sentiment of text 1: {'neg': 0.256, 'neu': 0.744, 'pos': 0.0, 'compound': -0.4767}
Sentiment of text 2: {'neg': 0.215, 'neu': 0.785, 'pos': 0.0, 'compound': -0.552}


In [106]:
# Install and import nltk
!pip install nltk
import nltk

# Download the lexicon
nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Sandr\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [111]:
 #Import the lexicon 
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Create an instance of SentimentIntensityAnalyzer
sent_analyzer = SentimentIntensityAnalyzer()

# Example
sentence = "Canada’s strategy to housing problems. Element Ireland can/should replicate?"
print(sent_analyzer.polarity_scores(sentence))

{'neg': 0.252, 'neu': 0.748, 'pos': 0.0, 'compound': -0.4019}


In [112]:
# What about this sentence with repeated exclamations
sentence_ = "About time there was a politician in our corner!!"
print(sent_analyzer.polarity_scores(sentence_))

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}


In [117]:
# What about this sentence with repeated exclamations
sentence_ = "Woo-hoo,(3rd time lucky)"
print(sent_analyzer.polarity_scores(sentence_))

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}


## VADER on Large Dataset

In [140]:

sentiment_data = pd.read_csv("Housing.csv")

sentiment_data.head(5)


,Councillors query long-term lease deal for 124 apartments at car park of former Regency Hotel
0,From Affordable Housing to Vulture Funds: An A...
1,So when are we pitching tents on the grounds o...
2,Northern Ireland Letting - Is criminal record ...
3,Affordable housing Bill 450K - Dan’s mix tape
4,Housing Prices Increase 11% in the Last Year


In [141]:
def format_data(data):

  last_col = str(data.columns[-1])
  first_col = str(data.columns[0])

  data.rename(columns = {last_col: 'comments', first_col: 'polarity'}, inplace=True) 

  # Change 0, 2, 4 to negative, neutral and positive
  labels = {0: 'negative', 2: 'neutral', 4: 'positive'}
  data['polarity'] = data['polarity'].map(labels)

  # Get only the two columns
  return data[['comments', 'polarity']]